# Run an experiment mixing StreamingLLM and Quantization
- [x] Run StreamingLLM
- [ ] Add Quantization to it

In [1]:
from dataclasses import dataclass
from contextlib import contextmanager

import torch
from torch import nn
from transformers import pipeline

from kvpress import BasePress, KnormPress, ScorerPress

In [2]:
# Load pipeline

device = "cuda:0"
ckpt = "Qwen/Qwen2.5-1.5B-Instruct"
attn_implementation = "flash_attention_2"
pipe = pipeline("kv-press-text-generation", model=ckpt, device=device, torch_dtype="auto", model_kwargs={"attn_implementation":attn_implementation})

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Device set to use cuda:0


In [3]:
# Load data

context = "In this step-by-step guide, you will learn how to create a new press in kvpress !"
question = "\nWhat is the purpose of this guide?"
tokens = pipe.tokenizer(context, return_tensors="pt").to(device)

In [4]:
compression_ratio = 0.6
press = KnormPress(compression_ratio)

with torch.no_grad():
    outputs_without_press = pipe.model(**tokens, output_hidden_states=True)

with torch.no_grad(), press(pipe.model):
    output_with_press = pipe.model(**tokens)

print(f"Cache shape w/o press: {outputs_without_press.past_key_values[0][0].shape}")
print(f"Cache shape w/ press:  {output_with_press.past_key_values[0][0].shape}\n")

# The `KVPressTextGenerationPipeline` simply applies the `press` as above on the context tokens (see `_forward` method for more details).
print(pipe(context, question=question, press=press)["answer"])

Cache shape w/o press: torch.Size([1, 2, 20, 128])
Cache shape w/ press:  torch.Size([1, 2, 8, 128])

The purpose of this guide is to provide step-by-step instructions on how to create a new press in KV (Kubernetes Versioning) version 1.18. The guide covers the necessary steps to set up a new press, including creating a


In [5]:
from transformers import QuantizedCacheConfig, QuantoQuantizedCache

config = QuantizedCacheConfig(nbits=4)
cache = QuantoQuantizedCache(config)

In [7]:
print(pipe(context, question=question, press=press, cache=cache)["answer"])

/ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/lib/python3.13/site-packages/torch/utils/cpp_extension.py:393: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (/opt/packages/nvidia/hpc_sdk//Linux_x86_64/22.9/compilers/bin/nvc++) is not compatible with the compiler Pytorch was
built with for this platform, which is g++ on linux. Please
use g++ to to compile your extension. Alternatively, you may
compile PyTorch from source using /opt/packages/nvidia/hpc_sdk//Linux_x86_64/22.9/compilers/bin/nvc++, and then you can also use
/opt/packages/nvidia/hpc_sdk//Linux_x86_64/22.9/compilers/bin/nvc++ to compile your extension.

See https://github.com/pytorch/pytorch/blob/master/CONTRIBUTING.md for help
with compiling PyTorch from source.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(WRONG_CO

RuntimeError: Error building extension 'quanto_cuda': [1/9] /opt/packages/cuda/v12.6.1/bin/nvcc --generate-dependencies-with-compile --dependency-output gptq_marlin_repack.cuda.o.d -ccbin /opt/packages/nvidia/hpc_sdk//Linux_x86_64/22.9/compilers/bin/nvc -DTORCH_EXTENSION_NAME=quanto_cuda -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -isystem /ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/lib/python3.13/site-packages/torch/include -isystem /ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/lib/python3.13/site-packages/torch/include/torch/csrc/api/include -isystem /ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/lib/python3.13/site-packages/torch/include/TH -isystem /ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/lib/python3.13/site-packages/torch/include/THC -isystem /opt/packages/cuda/v12.6.1/include -isystem /ocean/projects/cis240042p/hhirairi/.conda/envs/kvpress/incl